In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
#!pip uninstall sklearn-crfsuite
!pip install git+https://github.com/MeMartijn/updated-sklearn-crfsuite.git#egg=sklearn_crfsuite

  Cloning https://github.com/MeMartijn/updated-sklearn-crfsuite.git to /tmp/pip-install-9546ir8x/sklearn-crfsuite_7beb98833221412aa8a74d56f8f49665
  Running command git clone --filter=blob:none --quiet https://github.com/MeMartijn/updated-sklearn-crfsuite.git /tmp/pip-install-9546ir8x/sklearn-crfsuite_7beb98833221412aa8a74d56f8f49665
  Resolved https://github.com/MeMartijn/updated-sklearn-crfsuite.git to commit 675038761b4405f04691a83339d04903790e2b95
  Preparing metadata (setup.py) ... done


In [2]:
!pip install --upgrade pip
!pip install --upgrade setuptools

In [3]:
import pickle
with open("/content/drive/MyDrive/doc2vec_concat_128.pkl", 'rb') as f:
  dv_model = pickle.load(f)

In [4]:
with open("/content/drive/MyDrive/tagging_data.pkl", 'rb') as f:
  data = pickle.load(f)

In [5]:
def sent2features(embed):
  features ={}
  for i, e in enumerate(embed):
    features["v"+str(i)] = e
  return features

In [77]:
data

,emotion,dialogue_ID,utterance_ID,cause_span,t_conv,concat_conv
0,surprise,1,3,"[1, 3]","[{'utterance_ID': 1, 'text': 'Alright , so I a...","[{'utterance_ID': 1, 'data': 'Alright , so I a..."
1,surprise,1,4,"[1, 3, 4]","[{'utterance_ID': 1, 'text': 'Alright , so I a...","[{'utterance_ID': 1, 'data': 'Alright , so I a..."
2,anger,1,5,"[1, 3, 4]","[{'utterance_ID': 1, 'text': 'Alright , so I a...","[{'utterance_ID': 1, 'data': 'Alright , so I a..."
3,sadness,2,1,[1],"[{'utterance_ID': 1, 'text': 'I do not want to...","[{'utterance_ID': 1, 'data': 'I do not want to..."
4,surprise,2,3,[3],"[{'utterance_ID': 1, 'text': 'I do not want to...","[{'utterance_ID': 1, 'data': 'I do not want to..."
...,...,...,...,...,...,...
6756,anger,1360,11,[1],"[{'utterance_ID': 1, 'text': 'Wow ! Fortunatel...","[{'utterance_ID': 1, 'data': 'Wow ! Fortunatel..."
6757,joy,1371,1,[1],"[{'utterance_ID': 1, 'text': 'Ooh ! I am sorry...","[{'utterance_ID': 1, 'data': 'Ooh ! I am sorry..."
6758,anger,1371,6,[1],"[{'utterance_ID': 1, 'text': 'Ooh ! I am sorry...","[{'utterance_ID': 1, 'data': 'Ooh ! I am sorry..."
6759,surprise,1374,2,[1],"[{'utterance_ID': 1, 'text': 'I never sucked ,...","[{'utterance_ID': 1, 'data': 'I never sucked ,..."


In [78]:
def convert_df_to_model_data(data, embed_path):
  model_data = data.copy(deep=True)
  with open(embed_path, 'rb') as f:
    embedding = pickle.load(f)
  model_data["conv"] = model_data.apply(lambda x: [embedding[str(x["dialogue_ID"])+"utt"+str(y["utterance_ID"])] for y in x["t_conv"]], axis=1)
  model_data["oh_cause_spans"] = model_data.apply(lambda x: ["1" if i in x["cause_span"] else "0" for i in range(1, x["utterance_ID"]+1)], axis=1)
  model_data.drop(labels={"t_conv", "concat_conv", "cause_span"}, inplace=True, axis=1)
  return model_data

In [79]:
from sklearn.model_selection import train_test_split
import sklearn_crfsuite
from sklearn_crfsuite import metrics

In [84]:
def d2v_model_split_train(data, dv_model, algorithm="ap"):
  model_data = data.copy(deep=True)
  model_data["conv"] = model_data.apply(lambda x: [dv_model.dv[str(x["dialogue_ID"])+"utt"+str(y["utterance_ID"])] for y in x["concat_conv"]], axis=1)
  model_data["oh_cause_spans"] = model_data.apply(lambda x: ["1" if i in x["cause_span"] else "0" for i in range(1, x["utterance_ID"]+1)], axis=1)
  X_train, X_test, y_train, y_test = train_test_split(model_data["conv"], model_data["oh_cause_spans"], test_size=0.2, random_state=43)
  X_train = [sent2features(sent) for sent in X_train]
  y_train = [sent for sent in y_train]
  crf = sklearn_crfsuite.CRF(
    algorithm=algorithm,
    max_iterations=200,
    all_possible_transitions=True,
)
  crf.fit(X_train, y_train)
  X_test = [sent2features(sent) for sent in X_test]
  y_pred = crf.predict(X_test)
  print("For algorithm: ", algorithm)
  print("F1-Score:", metrics.flat_f1_score(y_test, y_pred,
                      average='weighted'))
  print("Precision: ",metrics.flat_precision_score(y_test, y_pred,
                      average='weighted'))
  print("Recall: ",metrics.flat_recall_score(y_test, y_pred,
                      average='weighted'))
  print("Whole Sequence Prediction Percentage: ",sklearn_crfsuite.metrics.sequence_accuracy_score(y_test, y_pred))

In [85]:
d2v_model_split_train(data, dv_model, algorithm="ap")

For algorithm:  ap
F1-Score: 0.8530215505124903
Precision:  0.8557521893053628
Recall:  0.8689752936411502
Whole Sequence Prediction Percentage:  0.3399852180339985


In [86]:
added_model_data = convert_df_to_model_data(data, "/content/drive/MyDrive/word2vec_add_128.pkl")
maxed_model_data = convert_df_to_model_data(data, "/content/drive/MyDrive/word2vec_max_128.pkl" )
avged_model_data = convert_df_to_model_data(data, "/content/drive/MyDrive/word2vec_avg_128.pkl")

In [91]:
def split_data_train_model(data, algorithm="ap"):
  X_train, X_test, y_train, y_test = train_test_split(data["conv"], data["oh_cause_spans"], test_size=0.2, random_state=43)
  X_train = [sent2features(sent) for sent in X_train]
  y_train = [sent for sent in y_train]
  crf = sklearn_crfsuite.CRF(
    algorithm=algorithm,
    max_iterations=200,
    all_possible_transitions=True,
)
  crf.fit(X_train, y_train)
  X_test = [sent2features(sent) for sent in X_test]
  y_pred = crf.predict(X_test)
  print("For algorithm:", algorithm)
  print("F1-Score:", metrics.flat_f1_score(y_test, y_pred,
                      average='weighted'))
  print("Precision:",metrics.flat_precision_score(y_test, y_pred,
                      average='weighted'))
  print("Recall:",metrics.flat_recall_score(y_test, y_pred,
                      average='weighted'))
  print("Whole Sequence Prediction Percentage:",sklearn_crfsuite.metrics.sequence_accuracy_score(y_test, y_pred))

In [92]:
print("Added Vectors")
split_data_train_model(added_model_data)
print("Averaged Vectors")
split_data_train_model(avged_model_data)
print("Maxed Vectors")
split_data_train_model(maxed_model_data)

Added Vectors
For algorithm: ap
F1-Score: 0.8530215505124903
Precision: 0.8557521893053628
Recall: 0.8689752936411502
Whole Sequence Prediction Percentage: 0.3399852180339985
Averaged Vectors
For algorithm: ap
F1-Score: 0.8530215505124903
Precision: 0.8557521893053628
Recall: 0.8689752936411502
Whole Sequence Prediction Percentage: 0.3399852180339985
Maxed Vectors
For algorithm: ap
F1-Score: 0.8530215505124903
Precision: 0.8557521893053628
Recall: 0.8689752936411502
Whole Sequence Prediction Percentage: 0.3399852180339985


In [93]:
print("Added Vectors")
split_data_train_model(added_model_data, algorithm="lbfgs")
print("Averaged Vectors")
split_data_train_model(avged_model_data, algorithm="lbfgs")
print("Maxed Vectors")
split_data_train_model(maxed_model_data, algorithm="lbfgs")

Added Vectors
For algorithm: lbfgs
F1-Score: 0.8568607466596021
Precision: 0.856506216655706
Recall: 0.8693803159173754
Whole Sequence Prediction Percentage: 0.32002956393200294
Averaged Vectors
For algorithm: lbfgs
F1-Score: 0.8568607466596021
Precision: 0.856506216655706
Recall: 0.8693803159173754
Whole Sequence Prediction Percentage: 0.32002956393200294
Maxed Vectors
For algorithm: lbfgs
F1-Score: 0.8568607466596021
Precision: 0.856506216655706
Recall: 0.8693803159173754
Whole Sequence Prediction Percentage: 0.32002956393200294


In [94]:
print("Added Vectors")
split_data_train_model(added_model_data, algorithm="l2sgd")
print("Averaged Vectors")
split_data_train_model(avged_model_data, algorithm="l2sgd")
print("Maxed Vectors")
split_data_train_model(maxed_model_data, algorithm="l2sgd")

Added Vectors
For algorithm: l2sgd
F1-Score: 0.8568607466596021
Precision: 0.856506216655706
Recall: 0.8693803159173754
Whole Sequence Prediction Percentage: 0.32002956393200294
Averaged Vectors
For algorithm: l2sgd
F1-Score: 0.8568607466596021
Precision: 0.856506216655706
Recall: 0.8693803159173754
Whole Sequence Prediction Percentage: 0.32002956393200294
Maxed Vectors
For algorithm: l2sgd
F1-Score: 0.8568607466596021
Precision: 0.856506216655706
Recall: 0.8693803159173754
Whole Sequence Prediction Percentage: 0.32002956393200294
